In [1]:
import pandas as pd
import numpy as np
# import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, \
        GradientBoostingClassifier, StackingClassifier, BaggingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier

from sklearn.preprocessing import label_binarize
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.metrics import balanced_accuracy_score, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

from sklearn.model_selection import cross_validate
from mlxtend.plotting import plot_learning_curves
import matplotlib.pyplot as plt
plt.style.use('ggplot')

from sklearn.metrics import confusion_matrix
import seaborn as sns

from sklearn.model_selection import train_test_split

import scikitplot as skplt

from keras.wrappers.scikit_learn import KerasClassifier


Using TensorFlow backend.


In [2]:
filedata = pd.read_pickle('./output/intermediate-data/filedata-mfccDeltaFeaturesforNN.pkl')
filedata_conventional = pd.read_pickle('./output/intermediate-data/filedata-librosaConventionalFeatures.pkl')

In [3]:
filedata.head()

,path,slice_file_name,fsID,start,end,salience,fold,classID,class,mfccDelta
0,data/UrbanSound8K/audio/fold3/195451-5-0-21.wav,195451-5-0-21.wav,195451,12.749062,16.749062,1,3,5,engine_idling,"[[[-70.071945, -0.11565272], [-67.96526, -0.11..."
1,data/UrbanSound8K/audio/fold3/112075-5-0-2.wav,112075-5-0-2.wav,112075,9.037355,13.037355,2,3,5,engine_idling,"[[[4.1909046, -0.59073174], [3.303571, -0.5907..."
2,data/UrbanSound8K/audio/fold3/98681-9-0-12.wav,98681-9-0-12.wav,98681,6.000000,10.000000,2,3,9,street_music,"[[[-237.0594, -0.58106667], [-233.21732, -0.58..."
3,data/UrbanSound8K/audio/fold3/165039-7-11-0.wav,165039-7-11-0.wav,165039,142.280541,146.280541,1,3,7,jackhammer,"[[[-125.816086, 4.1389656], [-115.63891, 4.138..."
4,data/UrbanSound8K/audio/fold3/103199-4-0-3.wav,103199-4-0-3.wav,103199,1.500000,5.500000,1,3,4,drilling,"[[[-40.986374, 0.17866911], [-40.307343, 0.178..."


In [4]:
filedata_conventional.head()

,path,slice_file_name,fsID,start,end,salience,fold,classID,class,mfccs,melSpec,chroma_stft,spectral_contrast_stft,tonnetz
0,data/UrbanSound8K/audio/fold3/195451-5-0-21.wav,195451-5-0-21.wav,195451,12.749062,16.749062,1,3,5,engine_idling,"[0.0, 1.0, 0.4437291, 0.59153306, 0.45830166, ...","[0.5128107, 0.9999999, 0.16205902, 0.07680624,...","[0.0, 0.098377876, 0.48215958, 0.82687426, 0.9...","[0.9848953492056421, 0.04095642911002232, 0.0,...","[0.0, 0.19750059281700416, 0.12797810698686948..."
1,data/UrbanSound8K/audio/fold3/112075-5-0-2.wav,112075-5-0-2.wav,112075,9.037355,13.037355,2,3,5,engine_idling,"[0.046528116, 1.0, 0.15096846, 0.42279488, 0.1...","[0.003973532, 0.038080055, 0.12683089, 0.74289...","[0.13973394, 0.17169072, 0.35754606, 0.9999974...","[0.999999882334079, 0.23397555009182935, 0.190...","[0.3170768093096214, 0.1630069259627819, 0.0, ..."
2,data/UrbanSound8K/audio/fold3/98681-9-0-12.wav,98681-9-0-12.wav,98681,6.000000,10.000000,2,3,9,street_music,"[0.0, 1.0, 0.566404, 0.75245655, 0.7023444, 0....","[0.07280028, 0.113039434, 0.05909347, 0.041520...","[0.37964487, 0.0, 0.37939143, 0.21343386, 0.39...","[0.0, 0.10896364935510458, 0.428032863770412, ...","[0.9881043882607795, 0.6284331586202113, 0.999..."
3,data/UrbanSound8K/audio/fold3/165039-7-11-0.wav,165039-7-11-0.wav,165039,142.280541,146.280541,1,3,7,jackhammer,"[0.0, 1.0, 0.5009636, 0.7033996, 0.4991666, 0....","[0.21449894, 0.92175645, 0.99999994, 0.7986757...","[0.75837964, 0.99999404, 0.63529855, 0.7780573...","[0.6674096293487833, 0.0, 0.3552743805022218, ...","[0.838027328973501, 0.6827474238051907, 0.0, 0..."
4,data/UrbanSound8K/audio/fold3/103199-4-0-3.wav,103199-4-0-3.wav,103199,1.500000,5.500000,1,3,4,drilling,"[0.3217654, 1.0, 0.0, 0.54690635, 0.47653884, ...","[0.0040449547, 0.0016131944, 0.001222729, 0.00...","[0.38465875, 0.2692769, 0.0, 0.26744914, 0.606...","[0.019249639681011364, 0.2645240905596094, 0.1...","[0.5379112844834538, 0.295805337385854, 0.0, 0..."


In [5]:
def get_cnn():
    num_filters = [24,32,64,128] 
    pool_size = (2, 2) 
    kernel_size = (3, 3)  
    input_shape = (40, 173, 2)
    num_classes = 10
    keras.backend.clear_session()
    
    model = keras.models.Sequential()
    model.add(keras.layers.Conv2D(24, kernel_size, input_shape=input_shape,
                padding="same"))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("relu"))
    model.add(keras.layers.MaxPooling2D(pool_size=pool_size))

    model.add(keras.layers.Conv2D(32, kernel_size,
                                  padding="same"))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("relu"))  
    model.add(keras.layers.MaxPooling2D(pool_size=pool_size))
    
    model.add(keras.layers.Conv2D(64, kernel_size,
                                  padding="same"))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("relu"))  
    model.add(keras.layers.MaxPooling2D(pool_size=pool_size))
    
    model.add(keras.layers.Conv2D(128, kernel_size,
                                  padding="same"))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("relu"))  

    model.add(keras.layers.GlobalMaxPooling2D())
    model.add(keras.layers.Dense(256, activation="relu"))
    model.add(keras.layers.Dense(num_classes, activation="softmax"))

    model.compile(optimizer=keras.optimizers.Adam(1e-3), 
        loss=keras.losses.SparseCategoricalCrossentropy(), 
        metrics=["accuracy"])
    return model

In [6]:
def load_model():
    # Create a basic model instance
    model = get_cnn()

    # Loads the weights
    model.load_weights('logs/2DMFCCwDelta/best_9.h5')

In [25]:
keras_one_vs_rest_nn_2d_classifier = KerasClassifier(load_model, verbose=0)

model_namelist = ['NN-2D'
                  ]

models = [
    OneVsRestClassifier(keras_one_vs_rest_nn_2d_classifier, n_jobs=-1)
    #keras_one_vs_rest_nn_2d_classifier
]

scoring = {'precision': make_scorer(precision_score, average='micro'), 
           'recall': make_scorer(recall_score, average='micro'), 
            'accuracy': make_scorer(accuracy_score), 
           'f1': make_scorer(f1_score, average='micro'),
           'roc_auc': make_scorer(roc_auc_score, average='micro')
          }

cv_result_entries = []

In [27]:
folds = np.array(list(range(1,11)))
kf = KFold(n_splits=10) 

for train_index, test_index in kf.split(folds):
    print('train_index: ' + str(train_index))
    print('test_index: ' + str(test_index))
    
    traindata = filedata[filedata['fold'].isin(list(folds[train_index]))]
    x_train = np.array(traindata['mfccDelta'].tolist())
    y_train = np.array(traindata['classID'].tolist())

    testdata = filedata[filedata['fold'] == folds[test_index][0]]
    x_test = np.array(testdata['mfccDelta'].tolist())
    y_test = np.array(testdata['classID'].tolist())
    
    #custom cv iterable
    myCViterator = []
    #for i in folds:
    #    print('fold, i: ' + str(i))
    #    trainIndices = filedata[ filedata['fold']!=i ].index.values.astype(int)
    #    testIndices =  filedata[ filedata['fold']==i ].index.values.astype(int)
    #    myCViterator.append( (trainIndices, testIndices) )
    #print('myCViterator: ' + str(myCViterator))
        

    for mod in models:
        metrics = cross_validate(
            mod,
            x_train,
            y_train,
            #cv=myCViterator, #fucking iterable does not work for some fucking reason
            scoring = scoring,
            return_train_score=False,
            n_jobs=-1
        )
        
        model_i = 0
        
        for key in metrics.keys():
            for fold_index, score in enumerate(metrics[key]):
                #print(model_i)
                #print(fold_index)
                #print(key)
                #print(score)
                cv_result_entries.append((model_namelist[model_i], fold_index, key, score))
        model_i += 1
    cv_results_df = pd.DataFrame(cv_result_entries)

train_index: [1 2 3 4 5 6 7 8 9]
test_index: [0]
fold, i: 1
fold, i: 2
fold, i: 3
fold, i: 4
fold, i: 5
fold, i: 6
fold, i: 7
fold, i: 8
fold, i: 9
fold, i: 10
train_index: [0 2 3 4 5 6 7 8 9]
test_index: [1]
fold, i: 1
fold, i: 2
fold, i: 3
fold, i: 4
fold, i: 5
fold, i: 6
fold, i: 7
fold, i: 8
fold, i: 9
fold, i: 10
train_index: [0 1 3 4 5 6 7 8 9]
test_index: [2]
fold, i: 1
fold, i: 2
fold, i: 3
fold, i: 4
fold, i: 5
fold, i: 6
fold, i: 7
fold, i: 8
fold, i: 9
fold, i: 10
train_index: [0 1 2 4 5 6 7 8 9]
test_index: [3]
fold, i: 1
fold, i: 2
fold, i: 3
fold, i: 4
fold, i: 5
fold, i: 6
fold, i: 7
fold, i: 8
fold, i: 9
fold, i: 10
train_index: [0 1 2 3 5 6 7 8 9]
test_index: [4]
fold, i: 1
fold, i: 2
fold, i: 3
fold, i: 4
fold, i: 5
fold, i: 6
fold, i: 7
fold, i: 8
fold, i: 9
fold, i: 10
train_index: [0 1 2 3 4 6 7 8 9]
test_index: [5]
fold, i: 1
fold, i: 2
fold, i: 3
fold, i: 4
fold, i: 5
fold, i: 6
fold, i: 7
fold, i: 8
fold, i: 9
fold, i: 10
train_index: [0 1 2 3 4 5 7 8 9]
test_in

KeyboardInterrupt: 

In [32]:
cv_results_df.loc[cv_results_df[2] == 'test_accuracy']

,0,1,2,3
20,NN-2D,0,test_accuracy,NaN
21,NN-2D,1,test_accuracy,NaN
22,NN-2D,2,test_accuracy,NaN
23,NN-2D,3,test_accuracy,NaN
24,NN-2D,4,test_accuracy,NaN
55,NN-2D,0,test_accuracy,NaN
56,NN-2D,1,test_accuracy,NaN
57,NN-2D,2,test_accuracy,NaN
58,NN-2D,3,test_accuracy,NaN
59,NN-2D,4,test_accuracy,NaN


In [31]:
cv_results_df

,0,1,2,3
0,NN-2D,0,fit_time,0.755018
1,NN-2D,1,fit_time,0.884453
2,NN-2D,2,fit_time,1.169800
3,NN-2D,3,fit_time,1.432906
4,NN-2D,4,fit_time,0.897267
...,...,...,...,...
310,NN-2D,0,test_roc_auc,NaN
311,NN-2D,1,test_roc_auc,NaN
312,NN-2D,2,test_roc_auc,NaN
313,NN-2D,3,test_roc_auc,NaN


In [19]:
#this code goes through and separates out each  elements of each feature array into separate columns
# DON'T CHANGE IT
mfccDelta = pd.DataFrame(filedata['mfccDelta'].iloc[x] for x in range(len(filedata)))
print(mfcsDelta)

X = pd.concat([mfccDelta], axis=1)

y = label_binarize(
      pd.DataFrame(filedata['classID'].iloc[x] for x in range(len(filedata))),
      classes=[0,1,2,3,4,5,6,7,8,9]
      )

ValueError: Must pass 2-d input

In [20]:
filedata['mfccDelta'].iloc[x] for x in range(len(filedata))

SyntaxError: invalid syntax (<ipython-input-20-9b90bb73b80c>, line 1)

In [ ]:
for mod in models:
        metrics = cross_validate(
            mod,
            x_train,
            y_train,
            #cv=myCViterator, #fucking iterable does not work for some fucking reason
            scoring = scoring,
            return_train_score=False,
            n_jobs=-1
        )
        
        model_i = 0
        
        for key in metrics.keys():
            for fold_index, score in enumerate(metrics[key]):
                #print(model_i)
                #print(fold_index)
                #print(key)
                #print(score)
                cv_result_entries.append((model_namelist[model_i], fold_index, key, score))
        model_i += 1
    cv_results_df = pd.DataFrame(cv_result_entries)